### Este Notebook esta enfocado en como conectar los estados con las reviews y Yelp

La columna `address` de la tabla principal `g_sitios` contiene informacion importante sobre como conectar las reviews de los estados y los negocios.

si se normaliza esta columna en una tabla aparte que contenga: la ciudad, la direccion, y codigo postal, mas la clave unica gmap_id, podemos agrupar por estados y crear un identificador unico.

con un eda sobre las categorias de los negocios en cada estado, podremos decidir que estados usar y cuales no, basandonos en datos y no en suposiciones.

In [85]:
import pandas as pd
import os

# Leemos la tabla g_sitios que tenemos hasta el momento.
g_sitios = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\data\google_sitios.parquet', engine='pyarrow')
g_sitios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2998428 entries, 0 to 3025010
Data columns (total 18 columns):
 #   Column          Dtype  
---  ------          -----  
 0   name            object 
 1   address         object 
 2   gmap_id         object 
 3   description     object 
 4   latitude        float64
 5   longitude       float64
 6   avg_rating      float64
 7   num_of_reviews  int64  
 8   price           object 
 9   state           object 
 10  url             object 
 11  monday          object 
 12  tuesday         object 
 13  wednesday       object 
 14  thursday        object 
 15  friday          object 
 16  saturday        object 
 17  sunday          object 
dtypes: float64(3), int64(1), object(14)
memory usage: 434.6+ MB


In [86]:
# Creamos la nueva tabla address que se conectara mediante gmap_id con la tabla principal.
address = g_sitios[['gmap_id','address', 'name']]
address.head()

,gmap_id,address,name
0,0x88f16e41928ff687:0x883dad4fd048e8f8,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",Porter Pharmacy
1,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",City Textile
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",San Soo Dang
3,0x80c2c89923b27a41:0x32041559418d447,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",Nova Fabrics
4,0x80c2c632f933b073:0xc31785961fe826a6,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",Nobel Textile Co


In [87]:
# Separar la columna 'address' en nuevas columnas
address_split = address['address'].str.split(',', n=3, expand=True)

# Cambiar el nombre de las columnas
address_split.columns = ['nombre', 'direccion', 'ciudad', 'cod.postal']

In [88]:
# Agregar la identificacion unica a las nuevas columnas separadas.
address = address[['gmap_id']].join(address_split)
address.head()

,gmap_id,nombre,direccion,ciudad,cod.postal
0,0x88f16e41928ff687:0x883dad4fd048e8f8,Porter Pharmacy,129 N Second St,Cochran,GA 31014
1,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,City Textile,3001 E Pico Blvd,Los Angeles,CA 90023
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,761 S Vermont Ave,Los Angeles,CA 90005
3,0x80c2c89923b27a41:0x32041559418d447,Nova Fabrics,2200 E 11th St,Los Angeles,CA 90021
4,0x80c2c632f933b073:0xc31785961fe826a6,Nobel Textile Co,719 E 9th St,Los Angeles,CA 90021


In [89]:
# Agregar una columna nueva llamada estados.
address['estado'] = None

# Listas de los 51 códigos postales y nombres de estados.
codigos_postales = [
    'CA', 'NY', 'IA', 'GA', 'FL', 'TX', 'LA', 'OR', 'WV', 'VA', 
    'AR', 'PA', 'NM', 'NC', 'TN', 'WI', 'NJ', 'IN', 'IL', 'DC', 
    'MD', 'ME', 'NE', 'WA', 'MI', 'OH', 'OK', 'MO', 'KS', 'UT', 
    'HI', 'NV', 'AZ', 'AL', 'CO', 'MA', 'ID', 'SC', 'RI', 'KY', 
    'AK', 'MT', 'MN', 'CT', 'MS', 'SD', 'WY', 'NH', 'DE', 'VT', 
    'ND'
]

nombres_estados = [
    'California', 'New York', 'Iowa', 'Georgia', 'Florida', 'Texas', 
    'Louisiana', 'Oregon', 'West Virginia', 'Virginia', 'Arkansas', 
    'Pennsylvania', 'New Mexico', 'North Carolina', 'Tennessee', 'Wisconsin', 
    'New Jersey', 'Indiana', 'Illinois', 'District of Columbia', 'Maryland', 
    'Maine', 'Nebraska', 'Washington', 'Michigan', 'Ohio', 'Oklahoma', 
    'Missouri', 'Kansas', 'Utah', 'Hawaii', 'Nevada', 'Arizona', 
    'Alabama', 'Colorado', 'Massachusetts', 'Idaho', 'South Carolina', 
    'Rhode Island', 'Kentucky', 'Alaska', 'Montana', 'Minnesota', 
    'Connecticut', 'Mississippi', 'South Dakota', 'Wyoming', 
    'New Hampshire', 'Delaware', 'Vermont', 'North Dakota'
]

In [90]:
# cod.postal contiene las letras del estado, podemos filtrar los estados rapidamente, a travez de un bucle entre codigo y nombres.
for codigo, estado in zip(codigos_postales, nombres_estados):
    address.loc[address['cod.postal'].str.contains(codigo, na=False), 'estado'] = estado

In [91]:
# El cod.postal puede ser nulo por falta de informacion ya que se corre la informacion hacia la izquierda.
# Para solucionar esto filtramos los cod.postales nulos que se corrieron hacia la izquierda
for codigo, estado in zip(codigos_postales, nombres_estados):
    address.loc[(address['cod.postal'].isnull()) & (address['ciudad'].str.contains(codigo, na=False)), 'estado'] = estado

In [92]:
# Ahora que estan corregidos las mayorias de los cod.postal algunos datos se cargaron al reves, conteniendo el cod en nombre y el nombre en cod.postal.
for estado in nombres_estados:
    address.loc[(address['estado'].isnull()) & (address['nombre'].str.contains(estado, na=False)), 'estado'] = estado

In [93]:
# Eliminar filas donde al menos 5 columnas son nulas
columnas = ['gmap_id', 'nombre', 'direccion', 'ciudad', 'cod.postal', 'estado']
address = address.dropna(thresh=len(columnas) - 1)

In [94]:
# Ultimas condicionales: el cod.postal no es nulo, pero su codigo aun se encuentra en la ciudad.
for codigo, estado in zip(codigos_postales, nombres_estados):
    address.loc[(address['cod.postal'].notnull()) & (address['ciudad'].str.contains(codigo, na=False)), 'estado'] = estado

# El cod.postal no es nulo, pero esta escrito el estado en la ciudad.  
for codigo, estado in zip(codigos_postales, nombres_estados):
    address.loc[(address['cod.postal'].notnull()) & (address['ciudad'].str.contains(estado, na=False)), 'estado'] = estado
    
# El cod.postal contiene escrito el estado y no un codigo.  
for codigo, estado in zip(codigos_postales, nombres_estados):
    address.loc[(address['cod.postal'].notnull()) & (address['cod.postal'].str.contains(estado, na=False)), 'estado'] = estado

In [95]:
# Terminada la normalizacion solo quedaron muy pocas filas donde no se puede saber su estado. Se eliminaran.
address = address.dropna(subset=['estado'])
address = address.drop(columns=['nombre'])
# Aplicamos un nuevo nombre al identificador unico, mas entendible.
address = address.rename(columns={'gmap_id': 'id_negocio'})

# Agregamos un identificador unico a cada estado para que se conecte con la tabla principal g_sitios.
address['id_estado'] = address['estado'].factorize()[0] + 1
address['id_estado'] = address['id_estado'].astype('int32')

In [96]:
# Mezclamos la informacion con la tabla principal.
g_sitios = g_sitios.rename(columns={'gmap_id': 'id_negocio'})
g_sitios = g_sitios.merge(address[['id_negocio', 'id_estado']], on='id_negocio', how='left')
g_sitios = g_sitios.drop(columns=['address'])
g_sitios = g_sitios.drop(columns=['state']) # Informacion estatica. No sirve

# Borrar filas con id_estado nulos, ya que no van a tener informacion para conectarse con reviews sin un estado.
g_sitios = g_sitios.dropna(subset=['id_estado'])
g_sitios['id_estado'] = g_sitios['id_estado'].astype('int32') # tipo de dato entero.

In [111]:
# Optimizando tipo de datos.
g_sitios['latitude'] = g_sitios['latitude'].astype('float32')
g_sitios['longitude'] = g_sitios['longitude'].astype('float32')
#  Extraemos las columnas latitud y longitud de g_sitios.
lat_long = g_sitios[['id_negocio', 'latitude', 'longitude']]

In [112]:
# Merge de lat_long a address usando id_negocio como clave.
address = address.merge(lat_long[['latitude','longitude','id_negocio']], on='id_negocio', how='left')
# Eliminar las columnas latitud y longitud de g_sitios
g_sitios = g_sitios.drop(columns=['latitude', 'longitude'])

In [115]:
# Terminado la informacion. guardamos el dataframe en parquet.
os.makedirs('../../../data', exist_ok=True)
address.to_parquet('../../../data/google_address.parquet', index=False)

# Muestra parcial de la tabla:
address.head()

,id_negocio,direccion,ciudad,cod.postal,estado,id_estado,latitude,longitude
0,0x88f16e41928ff687:0x883dad4fd048e8f8,129 N Second St,Cochran,GA 31014,Georgia,1,32.388302,-83.357101
1,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,3001 E Pico Blvd,Los Angeles,CA 90023,California,2,34.018890,-118.215286
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,761 S Vermont Ave,Los Angeles,CA 90005,California,2,34.058090,-118.292130
3,0x80c2c89923b27a41:0x32041559418d447,2200 E 11th St,Los Angeles,CA 90021,California,2,34.023670,-118.232933
4,0x80c2c632f933b073:0xc31785961fe826a6,719 E 9th St,Los Angeles,CA 90021,California,2,34.036694,-118.249420


In [116]:
# Muestra parcial de la tabla:
address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2888854 entries, 0 to 2888853
Data columns (total 8 columns):
 #   Column      Dtype  
---  ------      -----  
 0   id_negocio  object 
 1   direccion   object 
 2   ciudad      object 
 3   cod.postal  object 
 4   estado      object 
 5   id_estado   int32  
 6   latitude    float32
 7   longitude   float32
dtypes: float32(2), int32(1), object(5)
memory usage: 143.3+ MB


In [117]:
# Terminado la informacion. guardamos el dataframe principal actualizado en parquet.
os.makedirs('../../../data', exist_ok=True)
g_sitios.to_parquet('../../../data/google_sitios_act.parquet', index=False)

# Muestra parcial de la tabla:
g_sitios.head()

,name,id_negocio,description,avg_rating,num_of_reviews,price,url,monday,tuesday,wednesday,thursday,friday,saturday,sunday,id_estado
0,Porter Pharmacy,0x88f16e41928ff687:0x883dad4fd048e8f8,None,4.9,16,None,https://www.google.com/maps/place//data=!4m2!3...,8AM–6PM,8AM–6PM,8AM–12PM,8AM–6PM,8AM–6PM,8AM–12PM,Closed,1
1,City Textile,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,4.5,6,None,https://www.google.com/maps/place//data=!4m2!3...,No Disponible,No Disponible,No Disponible,No Disponible,No Disponible,No Disponible,No Disponible,2
2,San Soo Dang,0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,4.4,18,None,https://www.google.com/maps/place//data=!4m2!3...,Closed,6:30AM–6PM,6:30AM–6PM,6:30AM–6PM,6:30AM–6PM,6:30AM–6PM,7AM–12PM,2
3,Nova Fabrics,0x80c2c89923b27a41:0x32041559418d447,None,3.3,6,None,https://www.google.com/maps/place//data=!4m2!3...,9AM–5PM,9AM–5PM,9AM–5PM,9AM–5PM,9AM–5PM,Closed,Closed,2
4,Nobel Textile Co,0x80c2c632f933b073:0xc31785961fe826a6,None,4.3,7,None,https://www.google.com/maps/place//data=!4m2!3...,9AM–5PM,9AM–5PM,9AM–5PM,9AM–5PM,9AM–5PM,Closed,Closed,2


In [118]:
# Muestra parcial de la tabla:
g_sitios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2888854 entries, 0 to 2998427
Data columns (total 15 columns):
 #   Column          Dtype  
---  ------          -----  
 0   name            object 
 1   id_negocio      object 
 2   description     object 
 3   avg_rating      float64
 4   num_of_reviews  int64  
 5   price           object 
 6   url             object 
 7   monday          object 
 8   tuesday         object 
 9   wednesday       object 
 10  thursday        object 
 11  friday          object 
 12  saturday        object 
 13  sunday          object 
 14  id_estado       int32  
dtypes: float64(1), int32(1), int64(1), object(12)
memory usage: 341.6+ MB


Avanza hacia el Notebook 6. Donde se creara y ajustara la tabla horarios.